## Import modules

In [1]:
import numpy as np
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import os
import time

## Utils

In [2]:
def load_matrix(path: str, verbose: bool = True) -> np.ndarray:
    matrix = np.load(path)
    if verbose:
        print("Loaded matrix from", os.path.basename(path))
        print("Shape:", matrix.shape)
        print("Example:\n", matrix[:3], end="\n\n")
    return matrix

In [3]:
%cd ../../

/home/shkarupa/Documents/Semester6/Coursework/parallel-gradient-descent


## Generate data

In [4]:
!python demo/gen_data.py --help

Generate train and test data for linear regression model

Tool arguments:
  -h, --help          show this help message and exit
  -t , --n-train      number of train samples (default: 1000)
  -e , --n-eval       number of evaluation samples (default: 100)
  -f , --n-features   number of per sample features (default: 10)
  -s , --seed         seed that make data generation deterministic (default:
                      None)
  -o , --out-dir      path to directory where generated data will be stored
                      (default: ./data)


In [5]:
!python demo/gen_data.py --n-train 4000 --n-eval 400 --n-features 25 --seed 42 --out-dir data

## Look at data

In [6]:
!ls data

x_eval.npy  x_train.npy  y_eval.npy  y_train.npy


In [7]:
x_train = load_matrix("data/x_train.npy")
y_train = load_matrix("data/y_train.npy")

x_eval = load_matrix("data/x_eval.npy")
y_eval = load_matrix("data/y_eval.npy")

Loaded matrix from x_train.npy
Shape: (4000, 25)
Example:
 [[ 7.61510575e-04  5.10739396e-01  5.23892561e-01 -4.46711750e-01
  -4.73159956e-01  1.47420731e+00  1.64221552e+00 -7.58297348e-01
  -2.67756101e+00  1.91623842e-01  1.79700202e-01 -2.82382435e+00
   6.76787436e-01 -8.06064483e-01  2.39762175e-01 -2.54611772e+00
  -1.06302545e+00 -2.82332697e-02 -4.17779921e-01  2.52047770e+00
  -7.62234840e-01  1.43246353e-01  5.07992399e-01 -1.19967003e+00
  -5.29871112e-01]
 [-6.65335803e-01  3.06033282e-01  1.17493324e+00  1.71065867e+00
   6.63910328e-01 -2.87484170e-01 -2.43012131e-01 -9.81690716e-01
   5.85984180e-01 -8.13542573e-01  9.67778862e-01  2.07679659e+00
  -9.36234206e-01  1.34887047e+00  5.26148825e-01 -6.05668382e-01
   6.50604954e-01  1.55845726e-02  1.77088221e+00 -7.88585712e-01
   1.74056039e-01  6.23693210e-01 -3.06594525e-01 -1.89786746e-01
  -3.34701468e-01]
 [-8.81509965e-01  2.04751703e-02 -9.61155035e-01  9.37564014e-01
  -6.13402726e-01 -1.50291679e-01 -9.30338835

## Run Linear regression

In [8]:
!./build/bin/gradient_descent -h

Linear regression config:

CLI options:
  -h [ --help ]                        produce help message
  -c [ --config ] arg                  path to config file

Algorithm options:
  -i [ --input-path ] arg              path to input NPY file
  -t [ --target-path ] arg             path to target NPY file
  -e [ --eval-path ] arg               path to evaluation NPY file
  -o [ --out-path ] arg (=output.npy)  path to output NPY file
  -p [ --parallel ]                    wether to use parallel or serial SGD
  -n [ --num-epochs ] arg (=1000)      number of training epochs
  -l [ --lr ] arg (=0.001)             learning rate
  -w [ --weight-decay ] arg (=0.01)    L2 regularization lambda term
  --normalize                          wether to normalize input
  --num-threads arg (=11)              number of threads to use for parallel 
                                       SGD
  --num-step-epochs arg (=1)           number of epochs to compute in each 
                                       th

### SGD

In [9]:
!cat examples/serial_config.cfg

input-path = data/x_train.npy
target-path = data/y_train.npy
eval-path = data/x_eval.npy
out-path = output/serial_pred.npy
parallel = false
num-epochs = 10000
lr = 0.001
weight-decay = 0.01
normalize = true


In [10]:
!./build/bin/gradient_descent --config examples/serial_config.cfg

SGD: 763 ms


In [11]:
y_pred_sgd = load_matrix("output/serial_pred.npy", verbose=False)

In [12]:
print("MSE:", mean_squared_error(y_pred_sgd.squeeze(), y_eval))

MSE: 0.616383933259275


### Parallel SGD

In [13]:
!cat examples/parallel_config.cfg

input-path = data/x_train.npy
target-path = data/y_train.npy
eval-path = data/x_eval.npy
out-path = output/parallel_pred.npy
parallel = true
num-epochs = 10000
lr = 0.001
weight-decay = 0.01
normalize = true
num-threads = 11
num-step-epochs = 100


In [14]:
!./build/bin/gradient_descent --config examples/parallel_config.cfg

SGD: 347 ms


In [15]:
y_pred_parallel_sgd = load_matrix("output/parallel_pred.npy", verbose=False)

In [16]:
print("MSE:", mean_squared_error(y_pred_parallel_sgd, y_eval))

MSE: 0.6171191233879124


### Sklearn

In [17]:
regressor = make_pipeline(StandardScaler(), LinearRegression())
# regressor = make_pipeline(
#     StandardScaler(), 
#     SGDRegressor(alpha=0.01, learning_rate="constant", eta0=0.001),
# )

In [18]:
# start = time.perf_counter_ns()
regressor.fit(x_train, y_train.squeeze())
# end = time.perf_counter_ns()
# print("SGD:", (end - start) / 1e6, "ms")

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [19]:
y_pred = regressor.predict(x_eval)

In [20]:
print("MSE:", mean_squared_error(y_pred, y_eval))

MSE: 0.6155152676578428
